In [ ]:
from google.colab import output
output.enable_custom_widget_manager()


In [ ]:
! pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import transformers

print(transformers.__version__)

4.26.0


In [ ]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
 from datasets import load_dataset
 got_data = load_dataset("text", data_files="/content/got1.txt",split="train")

In [ ]:
got_data

Dataset({
    features: ['text'],
    num_rows: 20169
})

In [ ]:
from datasets import Dataset, DatasetDict,load_metric

data=got_data.train_test_split(test_size=0.1)

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 18152
    })
    test: Dataset({
        features: ['text'],
        num_rows: 2017
    })
})

In [ ]:
print(data["train"][1])
print(data["test"][3])

{'text': '"You hear it here and there, in inns and whorehouses." '}
{'text': 'obey." '}


In [ ]:
from transformers import AutoTokenizer

model_checkpoint="gpt2"
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets=data.map(tokenize_function,batched=True,num_proc=4,remove_columns="text")

#0:   0%|          | 0/5 [00:00<?, ?ba/s]

#2:   0%|          | 0/5 [00:00<?, ?ba/s]

#1:   0%|          | 0/5 [00:00<?, ?ba/s]

#3:   0%|          | 0/5 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets["train"][0]

{'input_ids': [1,
  5703,
  523,
  11,
  475,
  1521,
  30,
  3819,
  1450,
  547,
  7387,
  11,
  5443,
  11,
  7099,
  11,
  1521,
  373,
  3030,
  78,
  4558,
  75,
  262,
  1266,
  30,
  314,
  481,
  1560,
  220],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [ ]:
block_size=128

In [ ]:
 def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, though you could add padding instead if the model supports it
    # In this, as in all things, we advise you to follow your heart
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
clm_dataset=tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4
)

#0:   0%|          | 0/5 [00:00<?, ?ba/s]

#2:   0%|          | 0/5 [00:00<?, ?ba/s]

#1:   0%|          | 0/5 [00:00<?, ?ba/s]

#3:   0%|          | 0/5 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenizer.decode(clm_dataset["train"][0]["input_ids"])

'"Just so, but why? Other men were stronger, faster, younger, why was Syrio Forel the best? I will tell "You hear it here and there, in inns and whorehouses." hurt his leg?" happens. He has informants everywhere. His little birds, he calls them. One of his little birds heard about couldn\'t happen, it\'s a dream, I\'m having a nightmare. out. Come closer, boy. We can\'t see you behind those horses." Lord Jon, but he was full of oddments of kitchen gossip: the lord had been'

In [ ]:
from transformers import TFAutoModelForCausalLM

model= TFAutoModelForCausalLM.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
!pip install datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
train_set = model.prepare_tf_dataset(
    clm_dataset["train"],
    shuffle=True,
    batch_size=16,
)

validation_set = model.prepare_tf_dataset(
    clm_dataset["test"],
    shuffle=False,
    batch_size=16,
)

In [ ]:
from transformers import AdamWeightDecay,create_optimizer

optimizer=AdamWeightDecay(5e-5,weight_decay_rate=0.01)

In [ ]:
import tensorflow as tf


model.compile(optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:

from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, mode='min',restore_best_weights=True)


tensorboard_callback = TensorBoard(log_dir="./clm_model/logs")

PushToHubCallback = PushToHubCallback(
    output_dir= "./clm_model",
    tokenizer=tokenizer,
    hub_model_id="Amitesh007/text_generation-finetuned-gpt2"
)
callbacks=[tensorboard_callback,PushToHubCallback,early_stopping]

Cloning https://huggingface.co/Amitesh007/text_generation-finetuned-gpt2 into local empty directory.


Download file tf_model.h5:   0%|          | 16.5k/475M [00:00<?, ?B/s]

Download file logs/train/events.out.tfevents.1675091922.2978ed16ad1f.7739.0.v2:   1%|          | 7.43k/1.43M […

Download file logs/train/events.out.tfevents.1675095065.2978ed16ad1f.7739.6.v2:   0%|          | 1.44k/1.43M […

Download file logs/train/events.out.tfevents.1675157169.4a232d764816.7630.0.v2:   1%|1         | 16.5k/1.43M […

...

Clean file logs/validation/events.out.tfevents.1675092041.2978ed16ad1f.7739.1.v2: 100%|##########| 194/194 [00…

Clean file logs/validation/events.out.tfevents.1675163202.4a232d764816.7630.1.v2: 100%|##########| 193/193 [00…

Clean file logs/train/events.out.tfevents.1675156931.4a232d764816.5035.0.v2: 100%|##########| 40.0/40.0 [00:00…

Clean file logs/train/events.out.tfevents.1675096870.2978ed16ad1f.7739.8.v2:   0%|          | 1.00k/1.43M [00:…

Clean file tf_model.h5:   0%|          | 1.00k/475M [00:00<?, ?B/s]

In [ ]:
model.fit(train_set,validation_data=validation_set,epochs=2,callbacks=callbacks)

Epoch 1/2
181/181 [==============================] - 6579s 36s/step - loss: 3.9088 - val_loss: 3.6320
Epoch 2/2
122/181 [===================>..........] - ETA: 33:52 - loss: 3.6183

In [ ]:
model.push_to_hub("Amitesh007/text_generation-finetuned-gpt2",commit_message="end of training")

NameError: ignored

In [ ]:
import math

print(f"Perplexity: {math.exp(4.2344):.2f}")

Perplexity: 69.02


In [ ]:
test_sentence = " for the people and dragons are"

In [ ]:
tokenized = tokenizer(test_sentence, return_tensors="tf")

outputs = model.generate(**tokenized, max_length=100)

print(outputs)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tf.Tensor(
[[  329   262   661   290 20308   389  2636   290   262  2877   526   220
      1    40   423   645 24613   553  5966   531    13   366    40  4702
   3024  6099   526   220     1    40   423   645 24613   553 35754   531
     13   366    40   481   467   526   220     1    40   423   645 24613
    553  5966   531    13   366    40   481   467   526   220     1    40
    423   645 24613   553  5966   531    13   366    40   481   467   526
    220     1    40   423   645 24613   553  5966   531    13   366    40
    481   467   526   220     1    40   423   645 24613   553 35754   531
     13   366    40   481]], shape=(1, 100), dtype=int32)


In [ ]:
tokenizer.decode(outputs[0])

' for the people and dragons are dead and the living." "I have no thirst," Jon said. "I prefer hot beer." "I have no thirst," Ned said. "I will go." "I have no thirst," Jon said. "I will go." "I have no thirst," Jon said. "I will go." "I have no thirst," Jon said. "I will go." "I have no thirst," Ned said. "I will'